In [46]:
import os
import glob
import random
import cv2
import numpy as np
import multiprocessing
from functools import partial

In [47]:
image_folder = "images/gopro_hero_3_1080_w/*.jpg"
files = glob.glob(image_folder)

In [48]:
def process_image(filename, width, height):
    img = cv2.imread(filename)
    if img is None:  # Check if the file could be opened
        print("Image failed to load :", filename)
        return None
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (width, height), None)
    # If found, add object points, image points (after refining them)
    if ret:
        # Termination criteria for finding the sub pixel coordinates of corners (cornerSubPix)
        criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 60, 0.001)
        # Increase the pixel location accuracy
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        return corners2
    print("No chessboard : ", filename)
    os.remove(filename)
    return None

In [49]:
# internal corners of the grid
width, height = 10, 7
used_files = []
imgpoints = []  # 2d points in image plane
for f in files:
    pixel_points = process_image(f, width=width, height=height)
    if pixel_points is not None:
        used_files.append(f)
        imgpoints.append(pixel_points)
len(imgpoints)

42

In [55]:
# We assume the checker pattern is kept in the Z=0 plane and the camera is moved and rotated
# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((height*width, 3), np.float32)
objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2)

# Scale the 3d points to the actual dimensinos
squareSize = 23.55e-3  # meter
# objp = objp*squareSize
objpoints = []  # 3d point in  world space
[objpoints.append(objp) for i in range(len(imgpoints))]
len(objpoints)

42

In [56]:
runs = 10
mtx_sum = np.zeros((3,3))
dist_sum = np.zeros((1,5))
for r in range(runs):
    batch = 25
    idxs = random.sample(list(range(len(objpoints))), batch)
    batch_obj, batch_img = [], []
    for i in idxs:
        batch_obj.append(objpoints[i])
        batch_img.append(imgpoints[i])
    img = cv2.imread(files[0])
    rms, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(batch_obj, batch_img, img.shape[1::-1], None, None)
    mtx_sum += mtx
    dist_sum += dist
    print("RMS:", rms)
    # print("Camera Matrix:\n", mtx)
    # print("Distortion Coefficients:\n", dist.ravel())

RMS: 0.4073708165731539
RMS: 0.426879810209654
RMS: 0.44226250627703256
RMS: 0.4579288798846129
RMS: 0.45436362691285515
RMS: 0.38294767403745555
RMS: 0.7268607385219132
RMS: 0.711484980121665
RMS: 0.41850559660082043
RMS: 0.3647333298336603


In [57]:
mtx = mtx_sum/runs
dist = dist_sum/runs
print("Camera Matrix:\n", mtx)
print("Distortion Coefficients:\n", dist.ravel())

Camera Matrix:
 [[860.65480358   0.         950.2068911 ]
 [  0.         878.70893136 499.31480619]
 [  0.           0.           1.        ]]
Distortion Coefficients:
 [-2.69952989e-01  1.06840266e-01 -3.42939036e-05  2.35622423e-04
 -2.36067936e-02]


In [58]:
errors = np.zeros(batch)
for i in range(len(rvecs)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    errors[i] = error

In [59]:
for i in reversed(np.argsort(errors)):
    full_idx = idxs[i]
    print(files[full_idx], errors[i])

images/gopro_hero_3_1080_w\gopro_4_2628.jpg 61.321284497187634
images/gopro_hero_3_1080_w\gopro_3_679.jpg 59.19448464272196
images/gopro_hero_3_1080_w\gopro_2_1766.jpg 51.11888916779523
images/gopro_hero_3_1080_w\gopro_2_419.jpg 50.8375301470605
images/gopro_hero_3_1080_w\gopro_2_299.jpg 47.817093670010955
images/gopro_hero_3_1080_w\gopro_2_778.jpg 44.82350608722088
images/gopro_hero_3_1080_w\gopro_4_2411.jpg 39.211916749623335
images/gopro_hero_3_1080_w\gopro_1_352.jpg 38.28087773570755
images/gopro_hero_3_1080_w\gopro_1_359.jpg 34.37325189007433
images/gopro_hero_3_1080_w\gopro_2_1410.jpg 33.18907664760837
images/gopro_hero_3_1080_w\gopro_3_198.jpg 32.66517738420985
images/gopro_hero_3_1080_w\gopro_2_1023.jpg 32.34055380648357
images/gopro_hero_3_1080_w\gopro_4_2329.jpg 32.18884467555605
images/gopro_hero_3_1080_w\gopro_2_812.jpg 30.19651765688173
images/gopro_hero_3_1080_w\gopro_4_2071.jpg 28.08040046239429
images/gopro_hero_3_1080_w\gopro_2_265.jpg 27.788238651196682
images/gopro_h